In [159]:
import pandas as pd
import sqlite3
import sqlalchemy 

try:
    conn = sqlite3.connect("domains.db")    
except Exception as e:
    print(e)

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f"Table Name : {cursor.fetchall()}")

# Задаем df для более удобной работы
df = pd.read_sql_query('SELECT * FROM domains', conn)
df1 = pd.read_sql_query('SELECT * FROM rules', conn)
conn.close()

Table Name : [('domains',), ('rules',)]


In [160]:
df

,project_id,name
0,1234,api.alt-epg-dev.xxx.com
1,1234,00e11ed5-9e58-4bb1-8f65-244ee03861db.static.de...
2,1234,auth.fine.xxx.com
3,1234,07c3da1a-027d-4b30-8fdb-0b847963eef8.static.de...
4,1234,backend.beta.xxx.com
...,...,...
178,5678,9hdfdrv3os1j.sub.yyy.com
179,5678,ekfk2vivyyys.sub.yyy.com
180,5678,7namagxj3u1z.sub.yyy.com
181,5678,kbacb5r7vbuv.sub.yyy.com


In [163]:
df1

,project_id,regexp


In [170]:
# Порог возникновения
min_count_threshold = 2 

for project_id, group in df.groupby('project_id'):
    subdomain_counter = Counter()
    
    # Перебираем столбец «name» и разделяем каждый домен на поддомены
    for name in group['name']:
        subdomains = re.split(r'\.', name)
        
        # Считаем субдомены
        subdomain_counter.update(subdomains)
    
    # Удалить поддомены с количеством меньше порога и сортируем их 
    filtered_subdomain_counter = {subdomain: count for subdomain, count in subdomain_counter.items() if count >= min_count_threshold}
    filtered_subdomain_counter.pop('com')
    filtered_subdomain_counter = dict(sorted(filtered_subdomain_counter.items(), key=lambda x: x[1], reverse=False))
    
    # Создайте шаблон регулярного выражения, соответствующий поддоменам
    pattern = '|'.join(map(re.escape, list(filtered_subdomain_counter.keys())[0:-1]))

    # Создаем окончательный шаблон
    final_pattern = f'(?:{pattern}).*?{list(filtered_subdomain_counter.keys())[-1]}.com'
    
    # Сохраняем шаблон в словаре
    subdomain_counters[project_id] = final_pattern

print(subdomain_counters)

{'1234': '(?:hs\\-beta|files|strapi|prom|app|fine|mon|static|developer).*?xxx.com', '5678': '(?:xyz|sub).*?yyy.com'}


In [171]:
import pandas as pd

data = {
    'project_id': list(subdomain_counters.keys()),
    'regexp': list(subdomain_counters.values())}

df1 = pd.DataFrame(data)


In [172]:
df1

,project_id,regexp
0,1234,(?:hs\-beta|files|strapi|prom|app|fine|mon|sta...
1,5678,(?:xyz|sub).*?yyy.com


In [173]:
# Объедините шаблоны для разных идентификаторов проекта, используя '|' 
combined_regex = '|'.join(subdomain_counters.values())

# Примените объединенное регулярное выражение к DataFrame и извлеките все совпадения
matches = df['name'].str.findall(combined_regex)

# Отображение совпадений для каждой строки
for index, match_list in enumerate(matches):
    print(f"Строка {index}: {match_list}")

Строка 0: []
Строка 1: ['static.developer.xxx.com']
Строка 2: ['fine.xxx.com']
Строка 3: ['static.developer.xxx.com']
Строка 4: []
Строка 5: ['static.developer.xxx.com']
Строка 6: ['fine.xxx.com']
Строка 7: ['static.developer.xxx.com']
Строка 8: ['hs-beta.xxx.com']
Строка 9: ['static.developer.xxx.com']
Строка 10: ['files.hs-beta.xxx.com']
Строка 11: ['static.developer.xxx.com']
Строка 12: ['files.sm-beta.xxx.com']
Строка 13: ['static.developer.xxx.com']
Строка 14: ['mon.xxx.com']
Строка 15: ['static.developer.xxx.com']
Строка 16: ['mon.xxx.com']
Строка 17: ['static.developer.xxx.com']
Строка 18: []
Строка 19: ['static.developer.xxx.com']
Строка 20: []
Строка 21: ['static.developer.xxx.com']
Строка 22: []
Строка 23: ['static.developer.xxx.com']
Строка 24: ['app.xxx.com']
Строка 25: ['static.developer.xxx.com']
Строка 26: []
Строка 27: ['static.developer.xxx.com']
Строка 28: []
Строка 29: ['static.developer.xxx.com']
Строка 30: []
Строка 31: ['static.developer.xxx.com']
Строка 32: []
Ст